# RATIO 2019 - Benchmarking Workshop

PyTorch + Transformers

```bash
conda activate argmining19-ssc
pip install transformers
pip install future  # for torch.utils.tensorboard
pip install tensorboardX
```

In [1]:
import datetime
import json
import logging
import os
import pickle
import random
import time
import warnings
from functools import partial
from multiprocessing import Pool, cpu_count

import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

#from mxnet.gluon.data import Dataset, SimpleDataset

import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from torch.utils.tensorboard import SummaryWriter
# from tensorboardX import SummaryWriter
from transformers import AdamW, WarmupLinearSchedule

from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score)
from sklearn.model_selection import train_test_split
from sklearn import utils
from tqdm import tqdm, trange

I1202 10:12:03.844632 139647375513408 file_utils.py:39] PyTorch version 1.1.0 available.
I1202 10:12:03.896306 139647375513408 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
%matplotlib inline

In [3]:
warnings.filterwarnings('ignore')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

logger = logging.getLogger("NB: pytorch-BERT")

In [4]:
# set repeatable random state
np.random.seed(100)
random.seed(100)
# https://pytorch.org/docs/stable/notes/randomness.html
_ = torch.manual_seed(0)

In [5]:
# apply progress bars for pandas .apply() -> .progress_apply()
tqdm.pandas()

In [6]:
# make tqdm jupyter friendly
from tqdm import tqdm_notebook as tqdm
# for .progress_apply() we have to hack it like this?
tqdm().pandas()

In [7]:
class Timer:
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.time_start = time.time()

    def __exit__(self, *exc):
        time_end = time.time()
        time_delta = datetime.timedelta(seconds=(time_end - self.time_start))
        if self.name:
            print(("Time for [{}]: {}".format(self.name, time_delta)))
        else:
            print(("Time: {}".format(time_delta)))

# Task 1 - Same Side Classification

In [8]:
load_new = False
# store tagged data in pickle object

more_tests = False
# whether to compute various things

In [9]:
data_cross_path = 'data/same-side-classification/cross-topic/{}.csv'
data_within_path = 'data/same-side-classification/within-topic/{}.csv'
new_within_test = 'data/same-side-classification/within-topic/within_test.csv'

### Load within-topics and cross-topics data

In [10]:
if load_new:
    with Timer("read cross"):
        cross_traindev_df = pd.read_csv(data_cross_path.format('training'),
                                        quotechar='"',
                                        quoting=csv.QUOTE_ALL,
                                        encoding='utf-8',
                                        escapechar='\\',
                                        doublequote=False,
                                        index_col='id')
        cross_test_df = pd.read_csv(data_cross_path.format('test'), index_col='id')

    with Timer("read within"):
        within_traindev_df = pd.read_csv(data_within_path.format('training'),
                                         quotechar='"',
                                         quoting=csv.QUOTE_ALL,
                                         encoding='utf-8',
                                         escapechar='\\',
                                         doublequote=False,
                                         index_col='id')
        # within_test_df = pd.read_csv(data_within_path.format('test'),
        #                              quotechar='"',
        #                              quoting=csv.QUOTE_ALL,
        #                              encoding='utf-8',
        #                              escapechar='\\',
        #                              doublequote=True,  # <-- change, "" as quote escape in text?
        #                              index_col='id')
        within_test_df = pd.read_csv(data_within_path.format('test'), index_col='id')

    with Timer("read new within"):
        new_within_test_df = pd.read_csv(new_within_test, index_col='id')

In [11]:
#! head -n 5 data/same-side-classification/within-topic/test.csv

In [12]:
#! head -n 5 data/same-side-classification/within-topic/within_test.csv

In [13]:
if load_new:
    # Adding a tag for the topics in focus: "gay marriage" and "abortion"
    def add_tag(row):
        title = row['topic'].lower().strip()
        if "abortion" in title:
            row['tag'] = 'abortion'
        elif "gay marriage"  in title:
            row['tag'] = 'gay marriage'
        else:
            row['tag'] = 'NA'
        return row


    with Timer("tag cross traindev"):
        cross_traindev_df = cross_traindev_df.progress_apply(add_tag, axis=1)
    with Timer("tag cross test"):
        cross_test_df = cross_test_df.progress_apply(add_tag, axis=1)

    with Timer("tag within traindev"):
        within_traindev_df = within_traindev_df.progress_apply(add_tag, axis=1)
    with Timer("tag within test"):
        within_test_df = within_test_df.progress_apply(add_tag, axis=1)
    with Timer("tag new within test"):
        new_within_test_df = new_within_test_df.progress_apply(add_tag, axis=1)

### Cache data pre-processing

In [14]:
FN_TAGGED = "data/same-side-classification/tagged_data.pkl"

In [15]:
if load_new:
    with open(FN_TAGGED, "wb") as fp:
        pickle.dump(cross_traindev_df, fp)
        pickle.dump(cross_test_df, fp)
        pickle.dump(within_traindev_df, fp)
        pickle.dump(within_test_df, fp)
        pickle.dump(new_within_test_df, fp)

In [16]:
with open(FN_TAGGED, "rb") as fp:
    cross_traindev_df = pickle.load(fp)
    cross_test_df = pickle.load(fp)
    within_traindev_df = pickle.load(fp)
    within_test_df = pickle.load(fp)
    new_within_test_df = pickle.load(fp)

### Get an overview about each dataset

In [17]:
# requires nltk  wordtokenize
# from nltk.tokenize import sent_tokenize, word_tokenize
# model uses BERT Tokenizer ...

def get_overview(df, task='same-side', class_name='is_same_side'):
    # Total instance numbers
    total = len(df)
    print("Task: ", task)
    print('=' * 40, '\n')

    print('Total instances: ', total)
    print('\n')

    print('For each topic:')
    for tag, tag_df in df.groupby(['tag']):
        print(tag, ': ', len(tag_df), ' instances')
        if class_name in df.columns:
            for is_same_side, side_df in tag_df.groupby([class_name]):
                print('\t\t', is_same_side, ': ', len(side_df), ' instances')
    print('\n')

    if class_name in df.columns:
        print('For each class value:')
        for class_value, class_df in df.groupby([class_name]):
            print(class_value, ': ', len(class_df), ' instances')
        print('\n')

    print('Unique argument1:', len(df['argument1'].unique()))
    print('Unique argument2:', len(df['argument2'].unique()))
    arguments = df['argument1'].values
    arguments = np.concatenate([arguments, df['argument2'].values])

    print('Unique total arguments:', len(set(list(arguments))), '\n')
    
    return

    print('-' * 40, '\n')

    arguments_length_lst = [
        len(word_tokenize(x)) for x in df['argument1'].values
    ]
    arguments_length_lst.extend(
        [len(word_tokenize(x)) for x in df['argument2'].values])
    print('Words:')
    print('\tshortest argument:', min(arguments_length_lst), ' words')
    print('\tlongest argument:', max(arguments_length_lst), ' words')
    print('\targument average length:', np.mean(arguments_length_lst),
          ' words')

    arguments_sent_length_lst = [
        len(sent_tokenize(x)) for x in df['argument1'].values
    ]
    arguments_sent_length_lst.extend(
        [len(sent_tokenize(x)) for x in df['argument2'].values])
    print('Sentences:')
    print('\tshortest argument:', min(arguments_sent_length_lst), ' sentences')
    print('\tlongest argument:', max(arguments_sent_length_lst), ' sentences')
    print('\targument average length:', np.mean(arguments_sent_length_lst),
          ' sentences')

In [18]:
with Timer("overview cross"):
    get_overview(cross_traindev_df)

Task:  same-side

Total instances:  61048


For each topic:
abortion :  61048  instances
		 False :  29853  instances
		 True :  31195  instances


For each class value:
False :  29853  instances
True :  31195  instances


Unique argument1: 7828
Unique argument2: 7806
Unique total arguments: 9361 

Time for [overview cross]: 0:00:00.182008


In [19]:
with Timer("overview within"):
    get_overview(within_traindev_df)

Task:  same-side

Total instances:  63903


For each topic:
abortion :  40840  instances
		 False :  20006  instances
		 True :  20834  instances
gay marriage :  23063  instances
		 False :  9786  instances
		 True :  13277  instances


For each class value:
False :  29792  instances
True :  34111  instances


Unique argument1: 10508
Unique argument2: 10453
Unique total arguments: 13574 

Time for [overview within]: 0:00:00.221472


##### Count raw length

In [20]:
if more_tests:
    def compute_arg_len(row):
        row['argument1_len'] = len(row['argument1'])
        row['argument2_len'] = len(row['argument2'])
        row['argument12_len_diff'] = row['argument1_len'] - row['argument2_len']
        row['argument12_len_diff_abs'] = np.abs(row['argument12_len_diff'])
        return row


    cross_traindev_df = cross_traindev_df.progress_apply(compute_arg_len, axis=1)
    within_traindev_df = within_traindev_df.progress_apply(compute_arg_len, axis=1)
    cross_test_df = cross_test_df.progress_apply(compute_arg_len, axis=1)
    within_test_df = within_test_df.progress_apply(compute_arg_len, axis=1)

    cross_traindev_df.describe()
    within_traindev_df.describe()
    within_test_df.describe()

##### Tokenize and count tokens

In [21]:
# BERT Tokenizer

# config_class, model_class, tokenizer_class = BertConfig, BertForSequenceClassification, BertTokenizer

if False:
    ctx = mx.cpu()
    _, vocabulary = nlp.model.get_model('bert_12_768_12',
                                        dataset_name='book_corpus_wiki_en_uncased',
                                        pretrained=True, ctx=ctx, use_pooler=True,
                                        use_decoder=False, use_classifier=False)
    bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)
    tokenizer = bert_tokenizer

if False:
    from nltk.tokenize import sent_tokenize, word_tokenize
    # nltk.download('punct')


    # tokenizer from BERT
    def tokenize_arguments(row):
        # tokenize
        row['argument1_tokens'] = tokenizer(row['argument1'])
        row['argument2_tokens'] = tokenizer(row['argument2'])

        # count tokens
        row['argument1_len'] = len(row['argument1_tokens'])
        row['argument2_len'] = len(row['argument2_tokens'])
        # token number diff
        row['argument12_len_diff'] = row['argument1_len'] - row['argument2_len']
        row['argument12_len_diff_abs'] = np.abs(row['argument12_len_diff'])
        return row


    cross_traindev_df = cross_traindev_df.progress_apply(tokenize_arguments, axis=1)
    within_traindev_df = within_traindev_df.progress_apply(tokenize_arguments, axis=1)
    cross_test_df = cross_test_df.progress_apply(tokenize_arguments, axis=1)
    within_test_df = within_test_df.progress_apply(tokenize_arguments, axis=1)

    cross_traindev_df.describe()
    within_traindev_df.describe()
    within_test_df.describe()

In [22]:
if more_tests:
    def plot_lengths(df, slicen=None, abs_diff=True, title=None):
        if df is None:
            print("no lengths to plot")
            return

        arg1_lens = df['argument1_len']
        arg2_lens = df['argument2_len']
        arg_diff_len = df['argument12_len_diff']

        if abs_diff:
            arg_diff_len = np.abs(arg_diff_len)

        if slicen is not None:
            arg1_lens = arg1_lens[slicen]
            arg2_lens = arg2_lens[slicen]
            arg_diff_len = arg_diff_len[slicen]

        x = np.arange(len(arg1_lens))  # arange/linspace

        plt.subplot(2, 1, 1)
        plt.plot(x, arg1_lens, label='argument1')  # Linie: '-', 'o-', '.-'
        plt.plot(x, arg2_lens, label='argument2')  # Linie: '-', 'o-', '.-'
        plt.legend()
        plt.title('Lengths of arguments' if not title else title)
        plt.ylabel('Lengths of arguments 1 and 2')

        plt.subplot(2, 1, 2)
        plt.plot(x, arg_diff_len)
        plt.xlabel('Index')
        plt.ylabel('Differences')

        plt.show()


    plot_lengths(within_traindev_df, slice(None, None, 500), title='Length of arguments within train/dev, every 500')
    plot_lengths(cross_traindev_df, slice(None, None, 500), title='Length of arguments cross train/dev, every 500')
    plot_lengths(within_test_df, slice(None, None, 1), title='Length of arguments within test')

## Train model - Baseline

### train dev set - 70% 30%

In [23]:
def get_train_test_sets(df, ratio=0.30, random_state=1):
    X = df[['argument1', 'argument2', 'argument1_id', 'argument2_id', 'topic']]
    y = df[['is_same_side']]

    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size=ratio,
                                                        random_state=random_state,
                                                        shuffle=True)
    return X_train, X_test, y_train, y_test

# Transformer

**_Base code from [gh:grenwi](https://github.com/grenwi/argmining19-same-side-classification)_**

### Loss etc.

- [BertForSequenceClassification](https://github.com/huggingface/transformers/blob/master/transformers/modeling_bert.py#L962)
- [BCEWithLogitsLoss](https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss)
- [transformers GLUE ..](https://github.com/huggingface/transformers/tree/master/examples#glue)
- 

In [24]:
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss, BCEWithLogitsLoss
from transformers import BertConfig, BertModel, BertPreTrainedModel

# https://huggingface.co/transformers/_modules/transformers/configuration_bert.html


# see: BertForSequenceClassification
class BertForSameSideClassification(BertPreTrainedModel):
    r"""
        **labels**: (`optional`) ``torch.LongTensor`` of shape ``(batch_size,)``:
            Labels for computing the sequence classification/regression loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
            If ``config.num_labels == 1`` a regression loss is computed (Mean-Square loss),
            If ``config.num_labels > 1`` a classification loss is computed (Cross-Entropy).
    Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
        **loss**: (`optional`, returned when ``labels`` is provided) ``torch.FloatTensor`` of shape ``(1,)``:
            Classification (or regression if config.num_labels==1) loss.
        **logits**: ``torch.FloatTensor`` of shape ``(batch_size, config.num_labels)``
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
            list of ``torch.FloatTensor`` (one for the output of each layer + the output of the embeddings)
            of shape ``(batch_size, sequence_length, hidden_size)``:
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        **attentions**: (`optional`, returned when ``config.output_attentions=True``)
            list of ``torch.FloatTensor`` (one for each layer) of shape ``(batch_size, num_heads, sequence_length, sequence_length)``:
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.
    Examples::
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]
    """

    # configClass = BERTSameSideConfig

    def __init__(self, config):
        super(BertForSameSideClassification, self).__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, self.config.num_labels)

        self.loss_kwargs = dict()

        if self.num_labels == 1:
            # regression
            self.loss_cls = MSELoss
            # self.loss_cls = BCEWithLogitsLoss
        else:
            self.loss_cls = CrossEntropyLoss

        self.init_weights()

    def forward(self,
                input_ids=None,
                attention_mask=None,
                token_type_ids=None,
                position_ids=None,
                head_mask=None,
                inputs_embeds=None,
                labels=None):

        # forward(input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None)
        # outputs = self.bert(input_ids,
        #                     attention_mask=attention_mask,
        #                     token_type_ids=token_type_ids,
        #                     position_ids=position_ids,
        #                     head_mask=head_mask,
        #                     inputs_embeds=inputs_embeds)
        # input_embeds only in newer version of transformers>=2.1.1 (in current master but not in pip)
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask)

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        # add hidden states and attention if they are here
        outputs = (logits, ) + outputs[2:]

        if labels is not None:
            loss_fct = self.loss_cls()

            if self.num_labels == 1:
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss = loss_fct(logits.view(-1, self.num_labels),
                                labels.view(-1))
            outputs = (loss, ) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)


class BertForSameSideBCEClassification(BertPreTrainedModel):
    r"""
        **labels**: (`optional`) ``torch.LongTensor`` of shape ``(batch_size,)``:
            Labels for computing the sequence classification/regression loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
            If ``config.num_labels == 1`` a regression loss is computed (Mean-Square loss),
            If ``config.num_labels > 1`` a classification loss is computed (Cross-Entropy).
    Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
        **loss**: (`optional`, returned when ``labels`` is provided) ``torch.FloatTensor`` of shape ``(1,)``:
            Classification (or regression if config.num_labels==1) loss.
        **logits**: ``torch.FloatTensor`` of shape ``(batch_size, config.num_labels)``
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
            list of ``torch.FloatTensor`` (one for the output of each layer + the output of the embeddings)
            of shape ``(batch_size, sequence_length, hidden_size)``:
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        **attentions**: (`optional`, returned when ``config.output_attentions=True``)
            list of ``torch.FloatTensor`` (one for each layer) of shape ``(batch_size, num_heads, sequence_length, sequence_length)``:
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.
    Examples::
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]
    """

    def __init__(self, config):
        super(BertForSameSideBCEClassification, self).__init__(config)
        self.num_labels = 1

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, self.num_labels)

        self.loss_kwargs = dict()
        self.loss_cls = BCEWithLogitsLoss

        self.init_weights()
        nn.init.normal(self.classifier.weight, 0, 0.01)

    def forward(self,
                input_ids=None,
                attention_mask=None,
                token_type_ids=None,
                position_ids=None,
                head_mask=None,
                inputs_embeds=None,
                labels=None):

        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask)

        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        outputs = (logits, ) + outputs[2:]

        if labels is not None:
            loss_fct = self.loss_cls()
            loss = loss_fct(logits.view(-1), labels.view(-1))
            outputs = (loss, ) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)


# both?
class BertForSameSideI2OBCEClassification(BertPreTrainedModel):
    def __init__(self, config):
        super(BertForSameSideI2OBCEClassification, self).__init__(config)
        self.num_labels = 1

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size * 2, self.num_labels)

        self.loss_kwargs = dict()
        self.loss_cls = BCEWithLogitsLoss

        self.init_weights()

    def forward(self,
                input_ids=None,
                attention_mask=None,
                token_type_ids=None,
                position_ids=None,
                head_mask=None,
                inputs_embeds=None,
                input_ids2=None,
                attention_mask2=None,
                token_type_ids2=None,
                position_ids2=None,
                head_mask2=None,
                inputs_embeds2=None,
                labels=None):

        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask)
        outputs2 = self.bert(input_ids2,
                             attention_mask=attention_mask2,
                             token_type_ids=token_type_ids2,
                             position_ids=position_ids2,
                             head_mask=head_mask2)

        pooled_output = outputs[1]
        pooled_output2 = outputs2[1]
        pooled_output2_both = torch.cat((pooled_output, pooled_output2), 1)

        pooled_output2_both = self.dropout(pooled_output2_both)
        logits = self.classifier(pooled_output2_both)
        outputs = (logits, ) + outputs[2:]
        # for second input? -- (hidden_states), (attentions)

        if labels is not None:
            loss_fct = self.loss_cls()
            loss = loss_fct(logits.view(-1), labels.view(-1))
            outputs = (loss, ) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [25]:
from torch import nn
from torch.nn import BCEWithLogitsLoss
from transformers import XLNetModel, XLNetPreTrainedModel
from transformers.modeling_utils import SequenceSummary


class XLNetForSameSideBCEClassification(XLNetPreTrainedModel):
    r"""
        **labels**: (`optional`) ``torch.LongTensor`` of shape ``(batch_size,)``:
            Labels for computing the sequence classification/regression loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
            With ``config.num_labels == 1`` a regression loss is computed (Mean-Square loss),
    Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
        **loss**: (`optional`, returned when ``labels`` is provided) ``torch.FloatTensor`` of shape ``(1,)``:
            Classification (or regression if config.num_labels==1) loss.
        **logits**: ``torch.FloatTensor`` of shape ``(batch_size, config.num_labels)``
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        **mems**: (`optional`, returned when ``config.mem_len > 0``)
            list of ``torch.FloatTensor`` (one for each layer):
            that contains pre-computed hidden-states (key and values in the attention blocks) as computed by the model
            if config.mem_len > 0 else tuple of None. Can be used to speed up sequential decoding and attend to longer context.
            See details in the docstring of the `mems` input above.
        **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
            list of ``torch.FloatTensor`` (one for the output of each layer + the output of the embeddings)
            of shape ``(batch_size, sequence_length, hidden_size)``:
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        **attentions**: (`optional`, returned when ``config.output_attentions=True``)
            list of ``torch.FloatTensor`` (one for each layer) of shape ``(batch_size, num_heads, sequence_length, sequence_length)``:
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.
    Examples::
        tokenizer = XLNetTokenizer.from_pretrained('xlnet-large-cased')
        model = XLNetForSequenceClassification.from_pretrained('xlnet-large-cased')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]
    """

    def __init__(self, config):
        super(XLNetForSameSideBCEClassification, self).__init__(config)
        self.num_labels = 1
        
        self.transformer = XLNetModel(config)
        self.sequence_summary = SequenceSummary(config)
        self.logits_proj = nn.Linear(config.d_model, self.num_labels)

        self.loss_kwargs = dict()
        self.loss_cls = BCEWithLogitsLoss

        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, mems=None, perm_mask=None, target_mapping=None,
                token_type_ids=None, input_mask=None, head_mask=None, inputs_embeds=None, labels=None):
        transformer_outputs = self.transformer(input_ids,
                                               attention_mask=attention_mask,
                                               mems=mems,
                                               perm_mask=perm_mask,
                                               target_mapping=target_mapping,
                                               token_type_ids=token_type_ids,
                                               input_mask=input_mask,
                                               head_mask=head_mask)
                                               # inputs_embeds=inputs_embeds
        output = transformer_outputs[0]

        output = self.sequence_summary(output)
        logits = self.logits_proj(output)

        outputs = (logits,) + transformer_outputs[1:]  # Keep mems, hidden states, attentions if there are in it

        if labels is not None:
            loss_fct = self.loss_cls()
            loss = loss_fct(logits.view(-1), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # return (loss), logits, (mems), (hidden states), (attentions)

In [26]:
from torch import nn
from torch.nn import BCEWithLogitsLoss
from transformers import DistilBertConfig, DistilBertModel
from transformers.modeling_distilbert import DistilBertPreTrainedModel


class DistilBertForSameSideBCEClassification(DistilBertPreTrainedModel):
    r"""
        **labels**: (`optional`) ``torch.LongTensor`` of shape ``(batch_size,)``:
            Labels for computing the sequence classification/regression loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
            If ``config.num_labels == 1`` a regression loss is computed (BCEWithLogitsLoss),
    Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
        **loss**: (`optional`, returned when ``labels`` is provided) ``torch.FloatTensor`` of shape ``(1,)``:
            Classification (or regression if config.num_labels==1) loss.
        **logits**: ``torch.FloatTensor`` of shape ``(batch_size, config.num_labels)``
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
            list of ``torch.FloatTensor`` (one for the output of each layer + the output of the embeddings)
            of shape ``(batch_size, sequence_length, hidden_size)``:
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        **attentions**: (`optional`, returned when ``config.output_attentions=True``)
            list of ``torch.FloatTensor`` (one for each layer) of shape ``(batch_size, num_heads, sequence_length, sequence_length)``:
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.
    Examples::
        tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]
    """
    def __init__(self, config):
        super(DistilBertForSameSideBCEClassification, self).__init__(config)
        self.num_labels = 1

        self.distilbert = DistilBertModel(config)
        self.pre_classifier = nn.Linear(config.dim, config.dim)
        self.classifier = nn.Linear(config.dim, self.num_labels)
        self.dropout = nn.Dropout(config.seq_classif_dropout)

        self.loss_kwargs = dict()
        self.loss_cls = BCEWithLogitsLoss

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, head_mask=None, inputs_embeds=None, labels=None, token_type_ids=None):
        # ignore token_type_ids
        distilbert_output = self.distilbert(input_ids=input_ids,
                                            attention_mask=attention_mask,
                                            head_mask=head_mask)
                                            # inputs_embeds=inputs_embeds
        hidden_state = distilbert_output[0]                    # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]                    # (bs, dim)
        pooled_output = self.pre_classifier(pooled_output)   # (bs, dim)
        pooled_output = nn.ReLU()(pooled_output)             # (bs, dim)
        pooled_output = self.dropout(pooled_output)         # (bs, dim)
        logits = self.classifier(pooled_output)              # (bs, dim)

        outputs = (logits,) + distilbert_output[1:]
        if labels is not None:
            loss_fct = self.loss_cls()
            loss = loss_fct(logits.view(-1), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [27]:
from torch import nn
from torch.nn import BCEWithLogitsLoss
from transformers import RobertaConfig, RobertaModel, BertPreTrainedModel
from transformers.modeling_roberta import RobertaClassificationHead
from transformers import ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP


class RobertaForSameSideBCEClassification(BertPreTrainedModel):
    r"""
        **labels**: (`optional`) ``torch.LongTensor`` of shape ``(batch_size,)``:
            Labels for computing the sequence classification/regression loss.
            Indices should be in ``[0, ..., config.num_labels]``.
            If ``config.num_labels == 1`` a regression loss is computed (Mean-Square loss),
    Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
        **loss**: (`optional`, returned when ``labels`` is provided) ``torch.FloatTensor`` of shape ``(1,)``:
            Classification (or regression if config.num_labels==1) loss.
        **logits**: ``torch.FloatTensor`` of shape ``(batch_size, config.num_labels)``
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
            list of ``torch.FloatTensor`` (one for the output of each layer + the output of the embeddings)
            of shape ``(batch_size, sequence_length, hidden_size)``:
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        **attentions**: (`optional`, returned when ``config.output_attentions=True``)
            list of ``torch.FloatTensor`` (one for each layer) of shape ``(batch_size, num_heads, sequence_length, sequence_length)``:
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.
    Examples::
        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        model = RobertaForSequenceClassification.from_pretrained('roberta-base')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]
    """
    config_class = RobertaConfig
    pretrained_model_archive_map = ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP
    base_model_prefix = "roberta"

    def __init__(self, config):
        super(RobertaForSameSideBCEClassification, self).__init__(config)
        self.num_labels = 1

        self.roberta = RobertaModel(config)
        self.classifier = RobertaClassificationHead(config)

        self.loss_kwargs = dict()
        self.loss_cls = BCEWithLogitsLoss
    
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None,
                labels=None):
        outputs = self.roberta(input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask)
                               # inputs_embeds=inputs_embeds
        sequence_output = outputs[0]
        logits = self.classifier(sequence_output)

        outputs = (logits,) + outputs[2:]
        if labels is not None:
            loss_fct = self.loss_cls()
            loss = loss_fct(logits.view(-1), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [28]:
from torch import nn
from torch.nn import BCEWithLogitsLoss
from transformers import XLMModel, XLMPreTrainedModel
from transformers.modeling_utils import SequenceSummary


class XLMForSameSideBCEClassification(XLMPreTrainedModel):
    r"""
        **labels**: (`optional`) ``torch.LongTensor`` of shape ``(batch_size,)``:
            Labels for computing the sequence classification/regression loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
            If ``config.num_labels == 1`` a regression loss is computed (BCEWithLogitsLoss),
    Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
        **loss**: (`optional`, returned when ``labels`` is provided) ``torch.FloatTensor`` of shape ``(1,)``:
            Classification (or regression if config.num_labels==1) loss.
        **logits**: ``torch.FloatTensor`` of shape ``(batch_size, config.num_labels)``
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
            list of ``torch.FloatTensor`` (one for the output of each layer + the output of the embeddings)
            of shape ``(batch_size, sequence_length, hidden_size)``:
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        **attentions**: (`optional`, returned when ``config.output_attentions=True``)
            list of ``torch.FloatTensor`` (one for each layer) of shape ``(batch_size, num_heads, sequence_length, sequence_length)``:
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.
    Examples::
        tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-en-2048')
        model = XLMForSequenceClassification.from_pretrained('xlm-mlm-en-2048')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]
    """
    def __init__(self, config):
        super(XLMForSameSideBCEClassification, self).__init__(config)
        self.num_labels = 1

        self.transformer = XLMModel(config)
        self.sequence_summary = SequenceSummary(config)

        self.loss_kwargs = dict()
        self.loss_cls = BCEWithLogitsLoss

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, langs=None, token_type_ids=None, position_ids=None,
                lengths=None, cache=None, head_mask=None, inputs_embeds=None, labels=None):
        transformer_outputs = self.transformer(input_ids,
                                               attention_mask=attention_mask,
                                               langs=langs,
                                               token_type_ids=token_type_ids,
                                               position_ids=position_ids,
                                               lengths=lengths, 
                                               cache=cache,
                                               head_mask=head_mask)
                                               # inputs_embeds=inputs_embeds

        output = transformer_outputs[0]
        logits = self.sequence_summary(output)

        outputs = (logits,) + transformer_outputs[1:]  # Keep new_mems and attention/hidden states if they are here

        if labels is not None:
            loss_fct = self.loss_cls()
            loss = loss_fct(logits.view(-1), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs

### Configs

In [29]:
args = {
    #: set to true tu auto-set some params
    'is_ss_bce': True,
    'is_i2o': False,
    'title-note': "First try: DistilBert SS BCE epi 512 trim from front",
    'title': None,
    'auto_adjust': True,

    #: model_type: (bert|bert-ss|bert-ss-bce|...)
    'model_type':  'distilbert-ss-bce',
    'model_name': 'distilbert-base-uncased',

    #: task_name: (binary|binary-bce)
    'task_name': 'binary-bce',

    #: output dirs
    'data_dir': 'data/transformers/',
    'cache_dir': 'cache/transformers/',
    # 'output_dir': 'outputs/transformers/',
    # 'output_dir': 'outputs/transformers/binary-label2-class',
    # 'output_dir': 'outputs/transformers/binary-label1-reg',
    'output_dir': 'outputs/transformers/binary-label1-class-bce-proepi',
    # 'log_dir': 'logs/transformers/',
    # 'log_dir': 'logs/transformers/binary-label2-class',
    # 'log_dir': 'logs/transformers/binary-label1-reg',
    'log_dir': 'logs/transformers/binary-label1-class-bce-proepi',

    'do_train': True,
    'do_eval': True,

    'fp16': False,
    'fp16_opt_level': 'O1',

    'max_seq_length': 512,
    #: truncate_end: (True|False) -- truncate longer inputs from start (False) or end (True)
    'truncate_end': False,
    #: num_labels: (1|2)
    'num_labels': 1,
    #: output_mode: (regression|classification) -- regression := float, classification := labels (multiple)
    'output_mode': 'regression',
    #: train batch_size: batch/max_seq_len: 6/512, 16/256, 32/128
    'train_batch_size': 6,
    #: eval batch_size can probably be slightly larger?
    'eval_batch_size': 32,  # 128

    'gradient_accumulation_steps': 1,
    'num_train_epochs': 3,
    'weight_decay': 0,      # hmm?
    'learning_rate': 5e-6,  # same as MXNet
    'adam_epsilon': 1e-9,   # same as MXNet
    'warmup_steps': 0,      # hmm?
    'max_grad_norm': 1.0,   # same as MXNet

    'logging_steps': 50,
    'eval_steps': 3000,
    'evaluate_during_training': True,
    #: save_steps may need to be larger for smaller batch_sizes
    'save_steps': 3000,
    #: ?
    'eval_all_checkpoints': True,
    'overwrite_output_dir': False,
    #: cache it?
    'reprocess_input_data': False,
    'notes': 'SameSide argument classification task'
}

if args.get("auto_adjust", False):
    # set some params based on whether we compute same-side with BCE
    if args.get('is_ss_bce', False):
        if "-ss-bce" not in args["model_type"]:
            args["model_type"] = args["model_type"] + "-ss-bce"
        args["task_name"] = "binary-bce"
        args["num_labels"] = 1
        args["output_mode"] = "regression"

    # double input mode
    if args.get('is_i2o', False):
        assert args["model_type"].startswith("bert")
        args["model_type"] = args["model_type"] + "-i2o"
        args["task_name"] = args["task_name"] + "-i2o"
        if args["max_seq_length"] == 512:
            args["train_batch_size"] = 4  # TODO: later increase?
            args["eval_batch_size"] = 32  # min(6, args["eval_batch_size"])
            args["save_steps"] = 10000
            args["logging_steps"] = 5000

    # build output folder names
    title = args.get("title", None)
    if not title:
        title = args["task_name"]
        if args.get('is_i20', False):
            title += '-i2o'
    args["output_dir"] = "outputs/transformers/" + title
    args["log_dir"] = "logs/transformers/" + title

# computation device
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [30]:
from transformers import (
    WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer,
    XLMConfig, XLMForSequenceClassification, XLMTokenizer, XLNetConfig,
    XLNetForSequenceClassification, XLNetTokenizer, RobertaConfig,
    RobertaForSequenceClassification, RobertaTokenizer, DistilBertConfig,
    DistilBertForSequenceClassification, DistilBertTokenizer)

MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'bert-ss-bce':
    (BertConfig, BertForSameSideBCEClassification, BertTokenizer),
    'bert-ss-bce-i2o':
    (BertConfig, BertForSameSideI2OBCEClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlnet-ss-bce':
    (XLNetConfig, XLNetForSameSideBCEClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'xlm-ss-bce': (XLMConfig, XLMForSameSideBCEClassification, XLMTokenizer),
    'roberta':
    (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer),
    'roberta-ss-bce': (RobertaConfig, RobertaForSameSideBCEClassification,
                       RobertaTokenizer),
    'distilbert': (DistilBertConfig, DistilBertForSequenceClassification,
                   DistilBertTokenizer),
    'distilbert-ss-bce':
    (DistilBertConfig, DistilBertForSameSideBCEClassification,
     DistilBertTokenizer)
}

config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]

In [31]:
config = config_class.from_pretrained(args['model_name'],
                                      num_labels=args['num_labels'],
                                      finetuning_task=args['task_name'])
tokenizer = tokenizer_class.from_pretrained(args['model_name'])

I1202 10:12:12.506418 139647375513408 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at /home/ekoerner/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I1202 10:12:12.510771 139647375513408 configuration_utils.py:168] Model config {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "binary-bce",
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "n_heads": 12,
  "n_layers": 6,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1202 

In [32]:
model = model_class.from_pretrained(args['model_name'], num_labels=args['num_labels'])

I1202 10:12:13.619422 139647375513408 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at /home/ekoerner/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I1202 10:12:13.623948 139647375513408 configuration_utils.py:168] Model config {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "n_heads": 12,
  "n_layers": 6,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1202 10:12:14

In [33]:
model.to(device)

DistilBertForSameSideBCEClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (dropout): Dropout(p=0.1)
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropou

### Data handling

In [34]:
from transformers.data import InputExample
# from transformers.data import InputFeatures
from transformers.data import DataProcessor


# TODO: binary? [0, 1] ?
class SameSideProcessor(DataProcessor):
    """Processor for the sameside data set"""

    def __init__(self, trainset, devset):
        self.trainset = trainset
        self.devset = devset

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self.trainset, "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self.devset, "dev")

    def get_labels(self):
        """See base class."""
        return [False, True]

    def _create_examples(self, items, set_type):
        """Creates examples for the training and dev sets."""
        examples = []

        for (i, item) in enumerate(items):
            guid = "%s-%s" % (set_type, i)
            text_a = item[0]
            text_b = item[1]
            label = item[2]
            examples.append(
                InputExample(guid=guid,
                             text_a=text_a,
                             text_b=text_b,
                             label=label))
        return examples


class SameSideBinaryProcessor(DataProcessor):
    """Processor for the sameside data set, label is binary."""

    def __init__(self, trainset, devset):
        self.trainset = trainset
        self.devset = devset

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self.trainset, "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self.devset, "dev")

    def get_labels(self):
        """See base class."""
        return [0, 1]

    def _create_examples(self, items, set_type):
        """Creates examples for the training and dev sets."""
        examples = []

        for (i, item) in enumerate(items):
            guid = "%s-%s" % (set_type, i)
            text_a = item[0]
            text_b = item[1]
            label = 0 if not item[2] else 1
            examples.append(
                InputExample(guid=guid,
                             text_a=text_a,
                             text_b=text_b,
                             label=label))
        return examples


# different names compared to transformers.data.InputFeatures
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class InputI2OFeatures(object):
    """A single set of features of data for double input."""

    def __init__(self, input_feature1, input_feature2):
        self.input_ids = input_feature1.input_ids
        self.input_mask = input_feature1.input_mask
        self.segment_ids = input_feature1.segment_ids
        # shared label
        self.label_id = input_feature1.label_id
        self.input_ids2 = input_feature2.input_ids
        self.input_mask2 = input_feature2.input_mask
        self.segment_ids2 = input_feature2.segment_ids

In [35]:
def convert_example_to_feature(example,
                               label_map,
                               max_seq_length,
                               tokenizer,
                               output_mode,
                               cls_token_at_end,
                               cls_token,
                               sep_token,
                               pad_on_left,
                               pad_token=0,
                               sequence_a_segment_id=0,
                               sequence_b_segment_id=1,
                               cls_token_segment_id=1,
                               pad_token_segment_id=0,
                               mask_padding_with_zero=True,
                               truncate_end=True):

    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a,
                           tokens_b,
                           max_seq_length - 3,
                           from_end=truncate_end)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids:   0   0   0   0  0     0   0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = tokens_a + [sep_token]
    segment_ids = [sequence_a_segment_id] * len(tokens)

    if tokens_b:
        tokens += tokens_b + [sep_token]
        segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

    if cls_token_at_end:
        tokens = tokens + [cls_token]
        segment_ids = segment_ids + [cls_token_segment_id]
    else:
        tokens = [cls_token] + tokens
        segment_ids = [cls_token_segment_id] + segment_ids

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    if pad_on_left:
        input_ids = ([pad_token] * padding_length) + input_ids
        input_mask = ([0 if mask_padding_with_zero else 1] *
                      padding_length) + input_mask
        segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
    else:
        input_ids = input_ids + ([pad_token] * padding_length)
        input_mask = input_mask + ([0 if mask_padding_with_zero else 1] *
                                   padding_length)
        segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    if output_mode == "classification":
        label_id = label_map[example.label]
    elif output_mode == "regression":
        label_id = float(example.label)
    else:
        raise KeyError(output_mode)

    return InputFeatures(input_ids=input_ids,
                         input_mask=input_mask,
                         segment_ids=segment_ids,
                         label_id=label_id)


def convert_examples_to_features(examples,
                                 label_list,
                                 max_seq_length,
                                 tokenizer,
                                 output_mode,
                                 cls_token_at_end=False,
                                 pad_on_left=False,
                                 cls_token='[CLS]',
                                 sep_token='[SEP]',
                                 pad_token=0,
                                 sequence_a_segment_id=0,
                                 sequence_b_segment_id=1,
                                 cls_token_segment_id=1,
                                 pad_token_segment_id=0,
                                 mask_padding_with_zero=True,
                                 truncate_end=True):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label: i for i, label in enumerate(label_list)}

    fn_convert = partial(convert_example_to_feature,
                         label_map=label_map,
                         max_seq_length=max_seq_length,
                         tokenizer=tokenizer,
                         output_mode=output_mode,
                         cls_token_at_end=cls_token_at_end,
                         cls_token=cls_token,
                         sep_token=sep_token,
                         pad_on_left=pad_on_left,
                         cls_token_segment_id=cls_token_segment_id,
                         pad_token_segment_id=pad_token_segment_id,
                         truncate_end=truncate_end)

    process_count = cpu_count() - 2

    with Pool(process_count) as p:
        features = list(
            tqdm(p.imap(fn_convert, examples, chunksize=100),
                 total=len(examples)))

    return features


def convert_examples_to_features_i2o(examples,
                                     label_list,
                                     max_seq_length,
                                     tokenizer,
                                     output_mode,
                                     cls_token_at_end=False,
                                     pad_on_left=False,
                                     cls_token='[CLS]',
                                     sep_token='[SEP]',
                                     pad_token=0,
                                     sequence_a_segment_id=0,
                                     sequence_b_segment_id=1,
                                     cls_token_segment_id=1,
                                     pad_token_segment_id=0,
                                     mask_padding_with_zero=True):
    # currently only front and end, nothing with random etc.
    # we just re-use _truncate_seq_pair in both variants, everything else takes more work
    features1 = convert_examples_to_features(examples,
                                             label_list,
                                             max_seq_length,
                                             tokenizer,
                                             output_mode,
                                             cls_token_at_end=cls_token_at_end,
                                             pad_on_left=pad_on_left,
                                             cls_token=cls_token,
                                             sep_token=sep_token,
                                             pad_token=pad_token,
                                             sequence_a_segment_id=sequence_a_segment_id,
                                             sequence_b_segment_id=sequence_b_segment_id,
                                             cls_token_segment_id=cls_token_segment_id,
                                             pad_token_segment_id=pad_token_segment_id,
                                             mask_padding_with_zero=mask_padding_with_zero,
                                             truncate_end=True)
    features2 = convert_examples_to_features(examples,
                                             label_list,
                                             max_seq_length,
                                             tokenizer,
                                             output_mode,
                                             cls_token_at_end=cls_token_at_end,
                                             pad_on_left=pad_on_left,
                                             cls_token=cls_token,
                                             sep_token=sep_token,
                                             pad_token=pad_token,
                                             sequence_a_segment_id=sequence_a_segment_id,
                                             sequence_b_segment_id=sequence_b_segment_id,
                                             cls_token_segment_id=cls_token_segment_id,
                                             pad_token_segment_id=pad_token_segment_id,
                                             mask_padding_with_zero=mask_padding_with_zero,
                                             truncate_end=False)
    
    features = [InputI2OFeatures(f1, f2) for f1, f2 in tqdm(zip(features1, features2))]
    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length, from_end=True):
    """Truncates a sequence pair in place to the maximum length."""

    # from where to truncate (-1 (index) is from end, 0 is from the front)
    pop_pos = -1 if from_end else 0

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop(pop_pos)
        else:
            tokens_b.pop(pop_pos)

In [36]:
processors = {
    "binary": SameSideProcessor,
    "binary-bce": SameSideBinaryProcessor,
    "binary-bce-i2o": SameSideBinaryProcessor,
}

# not used?
output_modes = {
    "binary": "classification",
    "binary-bce": "regression",
    "binary-bce-i2o": "regression"
}

---

In [37]:
# 1. Getting train and dev data
with Timer("1 - test/train split"):
    X_train, X_dev, y_train, y_dev = get_train_test_sets(within_traindev_df, ratio=0.1)

Time for [1 - test/train split]: 0:00:00.013231


In [38]:
def df2ds(X, y):
    """Convert pandas data frames to training data set"""
    # join label to items
    df = X.merge(y, left_index=True, right_index=True)
    # filter neccessary columns
    df = df[["argument1", "argument2", "is_same_side"]]
    # skip id and convert to list
    ds = [i[1:] for i in df.itertuples()]
    return ds


def df2ds_rev(X, y):
    """Convert pandas data frames to training data set"""
    # join label to items
    df = X.merge(y, left_index=True, right_index=True)
    # filter neccessary columns
    df = df[["argument2", "argument1", "is_same_side"]]
    # skip id and convert to list
    ds = [i[1:] for i in df.itertuples()]
    return ds


def df2ds_test(X):
    # TODO: or keep id?
    df = df[["argument1", "argument2"]]
    ds = [i[1:] for i in df.itertuples()]
    return ds

In [39]:
with Timer("2 - convert train/dev sets input format"):
    task = args['task_name']

    ds_train = df2ds(X_train, y_train)
    ds_dev = df2ds(X_dev, y_dev)

# processor = processors[task](ds_train, ds_dev)
# label_list = processor.get_labels()
# num_labels = len(label_list)

Time for [2 - convert train/dev sets input format]: 0:00:00.055957


In [40]:
def load_and_cache_examples(ds_train, ds_dev, args, tokenizer, evaluate=False):
    task = args['task_name']
    processor = processors[task](ds_train, ds_dev)
    output_mode = args['output_mode']

    mode = 'dev' if evaluate else 'train'
    cached_features_file = os.path.join(
        args['cache_dir'],
        f"cached_{mode}_{args['model_name']}_{args['max_seq_length']}_{task}")

    if os.path.exists(cached_features_file) and not args.get(
            'reprocess_input_data', False):
        logger.info("Loading features from cached file %s",
                    cached_features_file)
        features = torch.load(cached_features_file)

    else:
        logger.info("Creating features from dataset file at %s",
                    args['data_dir'])
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(
            args['data_dir']) if evaluate else processor.get_train_examples(
                args['data_dir'])

        features = convert_examples_to_features(
            examples,
            label_list,
            args['max_seq_length'],
            tokenizer,
            output_mode,
            # xlnet has a cls token at the end
            cls_token_at_end=bool(args['model_type'] in ['xlnet']),
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id=2 if args['model_type'] in ['xlnet'] else 0,
            # pad on the left for xlnet
            pad_on_left=bool(args['model_type'] in ['xlnet']),
            pad_token_segment_id=4 if args['model_type'] in ['xlnet'] else 0,
            truncate_end=args['truncate_end'])

        logger.info("Saving features into cached file %s",
                    cached_features_file)
        torch.save(features, cached_features_file)

    all_input_ids = torch.tensor([f.input_ids for f in features],
                                 dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features],
                                  dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features],
                                   dtype=torch.long)
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features],
                                     dtype=torch.long)
    elif output_mode == "regression":
        all_label_ids = torch.tensor([f.label_id for f in features],
                                     dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                            all_label_ids)
    return dataset
        
        
def load_and_cache_examples_i2o(ds_train, ds_dev, args, tokenizer, evaluate=False):
    task = args['task_name']
    processor = processors[task](ds_train, ds_dev)
    output_mode = args['output_mode']

    mode = 'dev' if evaluate else 'train'
    cached_features_file = os.path.join(
        args['cache_dir'],
        f"cached_{mode}_{args['model_name']}_{args['max_seq_length']}_{task}")

    if os.path.exists(cached_features_file) and not args.get(
            'reprocess_input_data', False):
        logger.info("Loading features from cached file %s",
                    cached_features_file)
        features = torch.load(cached_features_file)

    else:
        logger.info("Creating features from dataset file at %s",
                    args['data_dir'])
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(
            args['data_dir']) if evaluate else processor.get_train_examples(
                args['data_dir'])

        features = convert_examples_to_features_i2o(
            examples,
            label_list,
            args['max_seq_length'],
            tokenizer,
            output_mode,
            # xlnet has a cls token at the end
            cls_token_at_end=bool(args['model_type'] in ['xlnet']),
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id=2 if args['model_type'] in ['xlnet'] else 0,
            # pad on the left for xlnet
            pad_on_left=bool(args['model_type'] in ['xlnet']),
            pad_token_segment_id=4 if args['model_type'] in ['xlnet'] else 0)

        logger.info("Saving features into cached file %s",
                    cached_features_file)
        torch.save(features, cached_features_file)

    all_input_ids = torch.tensor([f.input_ids for f in features],
                                 dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features],
                                  dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features],
                                   dtype=torch.long)
    all_input_ids2 = torch.tensor([f.input_ids2 for f in features],
                                 dtype=torch.long)
    all_input_mask2 = torch.tensor([f.input_mask2 for f in features],
                                  dtype=torch.long)
    all_segment_ids2 = torch.tensor([f.segment_ids2 for f in features],
                                   dtype=torch.long)
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features],
                                     dtype=torch.long)
    elif output_mode == "regression":
        all_label_ids = torch.tensor([f.label_id for f in features],
                                     dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                            all_input_ids2, all_input_mask2, all_segment_ids2,
                            all_label_ids)
    return dataset

---

In [41]:
# https://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.sigmoid.html
# https://stackoverflow.com/questions/43024745/applying-a-function-along-a-numpy-array


# def sigmoid(x):
#     return 1 / (1 + np.exp(-x))


from scipy.special import expit as sigmoid

In [42]:
from sklearn.metrics import mean_squared_error, matthews_corrcoef, confusion_matrix, accuracy_score, f1_score
from scipy.stats import pearsonr


def get_mismatched(labels, preds, args, ds_train, ds_dev):
    mismatched = labels != preds
    processor = processors[args['task_name']](ds_train, ds_dev)
    examples = processor.get_dev_examples(args['data_dir'])
    wrong = [i for (i, v) in zip(examples, mismatched) if v]

    return wrong


def get_eval_report(labels, preds, args, ds_train, ds_dev):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='binary')
    return {
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "acc": acc,
        "f1": f1
    }, get_mismatched(labels, preds, args, ds_train, ds_dev)


def compute_metrics(preds, labels, args, ds_train, ds_dev):
    assert len(preds) == len(labels)
    return get_eval_report(labels, preds, args, ds_train, ds_dev)

In [43]:
def write_eval_setup_args(args, prefix=""):
    eval_output_dir = args['output_dir']
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "a") as writer:
        writer.write("***** Experiments params {} *****\n".format(prefix))
        writer.write(json.dumps(args))
        writer.write("\n********************************\n\n")


def evaluate(model, tokenizer, args, ds_train, ds_dev, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args['output_dir']
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)

    results = {}
    EVAL_TASK = args['task_name']

    eval_dataset = load_and_cache_examples(ds_train,
                                           ds_dev,
                                           args,
                                           tokenizer,
                                           evaluate=True)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset,
                                 sampler=eval_sampler,
                                 batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                'input_ids':
                batch[0],
                'attention_mask':
                batch[1],
                # XLM don't use segment_ids
                'token_type_ids':
                batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,
                'labels':
                batch[3]
            }
            # Distilbert doesn't use token_type_ids
            if args['model_type'].split("-")[0] in ["distilbert"]:
                del inputs["token_type_ids"]
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        
        preds_ = logits.detach().cpu().numpy()
        out_label_ids_ = inputs['labels'].detach().cpu().numpy()
        
        # if args['output_mode'] == "classification" and args['num_labels'] == 1:
        #     # preds_ = preds_.sigmoid().round().astype('int32')
        #     out_label_ids_ = out_label_ids_.astype('float32')
        
        if preds is None:
            preds = preds_
            out_label_ids = out_label_ids_
        else:
            preds = np.append(preds, preds_, axis=0)
            out_label_ids = np.append(out_label_ids, out_label_ids_, axis=0)

    eval_loss = eval_loss / nb_eval_steps
    if args['output_mode'] == "classification":
        if args['num_labels'] == 1:
            preds = np.squeeze(preds)
        else:
            preds = np.argmax(preds, axis=1)
    elif args['output_mode'] == "regression":
        preds = np.squeeze(preds)
    
    # TODO: ?
    if args['num_labels'] == 1:
        preds = sigmoid(preds).round().astype('int32')
        out_label_ids = out_label_ids.astype('int32')

    try:
        result, wrong = compute_metrics(preds, out_label_ids, args, ds_train,
                                        ds_dev)
    except:
        result = wrong = None

    results.update(result)

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "a") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        writer.write("***** Eval results {} *****\n".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))
        writer.write("\n")

    return results, wrong


def evaluate_i2o(model, tokenizer, args, ds_train, ds_dev, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args['output_dir']
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)

    results = {}
    EVAL_TASK = args['task_name']

    eval_dataset = load_and_cache_examples_i2o(ds_train,
                                               ds_dev,
                                               args,
                                               tokenizer,
                                               evaluate=True)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset,
                                 sampler=eval_sampler,
                                 batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                'input_ids':
                batch[0],
                'attention_mask':
                batch[1],
                # XLM don't use segment_ids
                'token_type_ids':
                batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,
                'input_ids2':
                batch[3],
                'attention_mask2':
                batch[4],
                # XLM don't use segment_ids
                'token_type_ids2':
                batch[5] if args['model_type'] in ['bert', 'xlnet'] else None,
                'labels':
                batch[6]
            }
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        
        preds_ = logits.detach().cpu().numpy()
        out_label_ids_ = inputs['labels'].detach().cpu().numpy()
        
        if preds is None:
            preds = preds_
            out_label_ids = out_label_ids_
        else:
            preds = np.append(preds, preds_, axis=0)
            out_label_ids = np.append(out_label_ids, out_label_ids_, axis=0)

    eval_loss = eval_loss / nb_eval_steps
    if args['output_mode'] == "classification":
        if args['num_labels'] == 1:
            preds = np.squeeze(preds)
        else:
            preds = np.argmax(preds, axis=1)
    elif args['output_mode'] == "regression":
        preds = np.squeeze(preds)
    
    if args['num_labels'] == 1:
        preds = sigmoid(preds).round().astype('int32')
        out_label_ids = out_label_ids.astype('int32')

    try:
        result, wrong = compute_metrics(preds, out_label_ids, args, ds_train,
                                        ds_dev)
    except:
        result = wrong = None

    results.update(result)

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "a") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        writer.write("***** Eval results {} *****\n".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))
        writer.write("\n")

    return results, wrong

In [44]:
def get_train_output(model, tokenizer, args, ds_train, ds_dev, prefix="", evaluate=True):
    eval_dataset = load_and_cache_examples(ds_train,
                                           ds_dev,
                                           args,
                                           tokenizer,
                                           evaluate=evaluate)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset,
                                 sampler=eval_sampler,
                                 batch_size=args['eval_batch_size'])

    logger.info("***** Running model output gen {} *****".format(prefix))
    logger.info("  Evaluation mode = %s", evaluate)
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    eval_loss = 0.0
    nb_eval_steps = 0

    preds = None
    out_label_ids = None

    for batch in tqdm(eval_dataloader, desc="Get Model outputs"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                'input_ids':
                batch[0],
                'attention_mask':
                batch[1],
                # XLM don't use segment_ids
                'token_type_ids':
                batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,
                'labels':
                batch[3]
            }
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        
        preds_ = logits.detach().cpu().numpy()
        out_label_ids_ = inputs['labels'].detach().cpu().numpy()
        
        if preds is None:
            preds = preds_
            out_label_ids = out_label_ids_
        else:
            preds = np.append(preds, preds_, axis=0)
            out_label_ids = np.append(out_label_ids, out_label_ids_, axis=0)

    eval_loss = eval_loss / nb_eval_steps

    if args['output_mode'] == "classification":
        if args['num_labels'] == 1:
            preds = np.squeeze(preds)
        else:
            preds = np.argmax(preds, axis=1)
    elif args['output_mode'] == "regression":
        preds = np.squeeze(preds)
    
    if args['num_labels'] == 1:
        preds = sigmoid(preds).round().astype('int32')
        out_label_ids = out_label_ids.astype('int32')
        
    return preds, out_label_ids

In [45]:
def compute_train_acc(preds, labels, args):
    if args['output_mode'] == "classification":
        if args['num_labels'] == 1:
            preds = np.squeeze(preds)
        else:
            preds = np.argmax(preds, axis=1)
    elif args['output_mode'] == "regression":
        preds = np.squeeze(preds)
    if args['num_labels'] == 1:
        preds = sigmoid(preds).round().astype('int32')
        labels = labels.astype('int32')
    return accuracy_score(labels, preds)


def train(train_dataset, model, tokenizer, args, ds_train=None, ds_dev=None):
    tb_writer = SummaryWriter(args["log_dir"])

    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset,
                                  sampler=train_sampler,
                                  batch_size=args['train_batch_size'])

    t_total = len(train_dataloader) // args[
        'gradient_accumulation_steps'] * args['num_train_epochs']

    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [{
        'params': [
            p for n, p in model.named_parameters()
            if not any(nd in n for nd in no_decay)
        ],
        'weight_decay':
        args['weight_decay']
    }, {
        'params': [
            p for n, p in model.named_parameters()
            if any(nd in n for nd in no_decay)
        ],
        'weight_decay':
        0.0
    }]
    optimizer = AdamW(optimizer_grouped_parameters,
                      lr=args['learning_rate'],
                      eps=args['adam_epsilon'])
    scheduler = WarmupLinearSchedule(optimizer,
                                     warmup_steps=args['warmup_steps'],
                                     t_total=t_total)

    if args['fp16']:
        try:
            from apex import amp
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use fp16 training."
            )
        model, optimizer = amp.initialize(model,
                                          optimizer,
                                          opt_level=args['fp16_opt_level'])

    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args['num_train_epochs'])
    logger.info("  Total train batch size  = %d", args['train_batch_size'])
    logger.info("  Gradient Accumulation steps = %d",
                args['gradient_accumulation_steps'])
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")
    
    preds = out_label_ids = None  # inline acc computation

    for epoch_nr in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration {}/{}".format(epoch_nr + 1, args['num_train_epochs']))
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(device) for t in batch)
            inputs = {
                'input_ids':
                batch[0],
                'attention_mask':
                batch[1],
                # XLM don't use segment_ids
                'token_type_ids':
                batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,
                'labels':
                batch[3]
            }
            # Distilbert doesn't use token_type_ids
            if args['model_type'].split("-")[0] in ["distilbert"]:
                del inputs["token_type_ids"]
            outputs = model(**inputs)
            # model outputs are always tuple in pytorch-transformers (see doc)
            loss = outputs[0]
            # print("\rLoss: %f" % loss, end='')  # has no "real" meaning for me?

            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']

            if args['fp16']:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer),
                                               args['max_grad_norm'])

            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(),
                                               args['max_grad_norm'])

            tr_loss += loss.item()
            if (step + 1) % args['gradient_accumulation_steps'] == 0:

                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()

                global_step += 1

                if args['eval_steps'] > 0 and global_step % args[
                        'eval_steps'] == 0:
                    # Log metrics
                    # Only evaluate when single GPU otherwise metrics may not average well
                    if args['evaluate_during_training']:
                        results, _ = evaluate(model, tokenizer, args, ds_train,
                                              ds_dev)
                        for key, value in results.items():
                            tb_writer.add_scalar('eval_{}'.format(key), value,
                                                 global_step)

                if args['logging_steps'] > 0 and global_step % args[
                        'logging_steps'] == 0:

                    # compute acc (see eval)
                    preds_ = outputs[1].detach().cpu().numpy()  # logits
                    out_label_ids_ = inputs['labels'].detach().cpu().numpy()
                    if preds is None:
                        preds = preds_
                        out_label_ids = out_label_ids_
                    else:
                        preds = np.append(preds, preds_, axis=0)
                        out_label_ids = np.append(out_label_ids, out_label_ids_, axis=0)
                    acc = compute_train_acc(preds, out_label_ids, args)
                    cur_acc = compute_train_acc(preds_, out_label_ids_, args)
                    print("train acc: {:.3f} (cur acc: {:.3f})".format(acc, cur_acc), end="\r")

                    tb_writer.add_scalar('lr',
                                         scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar('acc', acc, global_step)
                    # tb_writer.add_pr_curve('pr_curve', out_label_ids_, preds_, global_step)
                    tb_writer.add_scalar('loss', (tr_loss - logging_loss) /
                                         args['logging_steps'], global_step)
                    logging_loss = tr_loss

                if args['save_steps'] > 0 and global_step % args[
                        'save_steps'] == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(
                        args['output_dir'],
                        'checkpoint-{}'.format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    # Take care of distributed/parallel training
                    model_to_save = model.module if hasattr(
                        model, 'module') else model
                    model_to_save.save_pretrained(output_dir)
                    logger.info("Saving model checkpoint to %s", output_dir)

    try:
        tb_writer.close()
    except Exception as ex:
        logger.exception("SummaryWriter.close() error?")

    return global_step, tr_loss / global_step


def train_i2o(train_dataset, model, tokenizer, args, ds_train=None, ds_dev=None):
    tb_writer = SummaryWriter(args["log_dir"])

    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset,
                                  sampler=train_sampler,
                                  batch_size=args['train_batch_size'])

    t_total = len(train_dataloader) // args[
        'gradient_accumulation_steps'] * args['num_train_epochs']

    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [{
        'params': [
            p for n, p in model.named_parameters()
            if not any(nd in n for nd in no_decay)
        ],
        'weight_decay':
        args['weight_decay']
    }, {
        'params': [
            p for n, p in model.named_parameters()
            if any(nd in n for nd in no_decay)
        ],
        'weight_decay':
        0.0
    }]
    optimizer = AdamW(optimizer_grouped_parameters,
                      lr=args['learning_rate'],
                      eps=args['adam_epsilon'])
    scheduler = WarmupLinearSchedule(optimizer,
                                     warmup_steps=args['warmup_steps'],
                                     t_total=t_total)

    if args['fp16']:
        try:
            from apex import amp
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use fp16 training."
            )
        model, optimizer = amp.initialize(model,
                                          optimizer,
                                          opt_level=args['fp16_opt_level'])

    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args['num_train_epochs'])
    logger.info("  Total train batch size  = %d", args['train_batch_size'])
    logger.info("  Gradient Accumulation steps = %d",
                args['gradient_accumulation_steps'])
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")

    for epoch_nr in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration {}/{}".format(epoch_nr + 1, args['num_train_epochs']))
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(device) for t in batch)
            inputs = {
                'input_ids':
                batch[0],
                'attention_mask':
                batch[1],
                # XLM don't use segment_ids
                'token_type_ids':
                batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,
                'input_ids2':
                batch[3],
                'attention_mask2':
                batch[4],
                # XLM don't use segment_ids
                'token_type_ids2':
                batch[5] if args['model_type'] in ['bert', 'xlnet'] else None,
                'labels':
                batch[6]
            }
            outputs = model(**inputs)
            # model outputs are always tuple in pytorch-transformers (see doc)
            loss = outputs[0]

            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']

            if args['fp16']:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer),
                                               args['max_grad_norm'])

            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(),
                                               args['max_grad_norm'])

            tr_loss += loss.item()
            if (step + 1) % args['gradient_accumulation_steps'] == 0:

                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()

                global_step += 1

                if args['logging_steps'] > 0 and global_step % args[
                        'logging_steps'] == 0:
                    # Log metrics
                    # Only evaluate when single GPU otherwise metrics may not average well
                    if args['evaluate_during_training']:
                        results, _ = evaluate_i2o(model, tokenizer, args, ds_train,
                                                  ds_dev)
                        for key, value in results.items():
                            tb_writer.add_scalar('eval_{}'.format(key), value,
                                                 global_step)
                    tb_writer.add_scalar('lr',
                                         scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar('loss', (tr_loss - logging_loss) /
                                         args['logging_steps'], global_step)
                    logging_loss = tr_loss

                if args['save_steps'] > 0 and global_step % args[
                        'save_steps'] == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(
                        args['output_dir'],
                        'checkpoint-{}'.format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    # Take care of distributed/parallel training
                    model_to_save = model.module if hasattr(
                        model, 'module') else model
                    model_to_save.save_pretrained(output_dir)
                    logger.info("Saving model checkpoint to %s", output_dir)

    try:
        tb_writer.close()
    except Exception as ex:
        logger.exception("SummaryWriter.close() error?")

    return global_step, tr_loss / global_step

---

In [46]:
# training
if args['do_train']:
    write_eval_setup_args(args)
    with Timer("3 - train (fine-tune) model"):
        if args.get('is_i2o', False):
            _ = evaluate_i2o(model, tokenizer, args, ds_train, ds_dev)
            train_dataset = load_and_cache_examples_i2o(ds_train, ds_dev, args, tokenizer)
            global_step, tr_loss = train_i2o(train_dataset, model, tokenizer, args, ds_train=ds_train, ds_dev=ds_dev)
            _ = evaluate_i2o(model, tokenizer, args, ds_train, ds_dev)

        else:
            train_dataset = load_and_cache_examples(ds_train, ds_dev, args, tokenizer)
            global_step, tr_loss = train(train_dataset, model, tokenizer, args, ds_train=ds_train, ds_dev=ds_dev)
            _ = evaluate(model, tokenizer, args, ds_train, ds_dev)

        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

I1202 10:12:46.397599 139647375513408 <ipython-input-40-fa36fc6f9a16>:14] Loading features from cached file cache/transformers/cached_train_distilbert-base-uncased_512_binary-bce
I1202 10:12:50.748721 139647375513408 <ipython-input-45-08f342e6a116>:60] ***** Running training *****
I1202 10:12:50.749628 139647375513408 <ipython-input-45-08f342e6a116>:61]   Num examples = 57512
I1202 10:12:50.750196 139647375513408 <ipython-input-45-08f342e6a116>:62]   Num Epochs = 3
I1202 10:12:50.750678 139647375513408 <ipython-input-45-08f342e6a116>:63]   Total train batch size  = 6
I1202 10:12:50.751140 139647375513408 <ipython-input-45-08f342e6a116>:65]   Gradient Accumulation steps = 1
I1202 10:12:50.751663 139647375513408 <ipython-input-45-08f342e6a116>:66]   Total optimization steps = 28758
Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

I1202 10:22:27.668914 139647375513408 <ipython-input-40-fa36fc6f9a16>:19] Creating features from dataset file at data/transformers/


I1202 10:22:32.624720 139647375513408 <ipython-input-40-fa36fc6f9a16>:42] Saving features into cached file cache/transformers/cached_dev_distilbert-base-uncased_512_binary-bce
I1202 10:22:35.358091 139647375513408 <ipython-input-43-1a70036edb06>:34] ***** Running evaluation  *****
I1202 10:22:35.359086 139647375513408 <ipython-input-43-1a70036edb06>:35]   Num examples = 6391
I1202 10:22:35.359692 139647375513408 <ipython-input-43-1a70036edb06>:36]   Batch size = 32


I1202 10:23:35.319556 139647375513408 <ipython-input-43-1a70036edb06>:105] ***** Eval results  *****
I1202 10:23:35.320501 139647375513408 <ipython-input-43-1a70036edb06>:108]   acc = 0.6240025035205758
I1202 10:23:35.321291 139647375513408 <ipython-input-43-1a70036edb06>:108]   f1 = 0.6448943401802866
I1202 10:23:35.322016 139647375513408 <ipython-input-43-1a70036edb06>:108]   fn = 1250
I1202 10:23:35.322731 139647375513408 <ipython-input-43-1a70036edb06>:108]   fp = 1153
I1202 10:23:35.323436 139647375513408 <ipython-input-43-1a70036edb06>:108]   mcc = 0.24568143894044497
I1202 10:23:35.324122 139647375513408 <ipython-input-43-1a70036edb06>:108]   tn = 1806
I1202 10:23:35.324613 139647375513408 <ipython-input-43-1a70036edb06>:108]   tp = 2182
I1202 10:23:35.328754 139647375513408 configuration_utils.py:71] Configuration saved in outputs/transformers/binary-bce/checkpoint-3000/config.json
I1202 10:23:35.501348 139647375513408 modeling_utils.py:205] Model weights saved in outputs/trans

I1202 10:33:13.530078 139647375513408 <ipython-input-40-fa36fc6f9a16>:14] Loading features from cached file cache/transformers/cached_dev_distilbert-base-uncased_512_binary-bce
I1202 10:33:13.967953 139647375513408 <ipython-input-43-1a70036edb06>:34] ***** Running evaluation  *****
I1202 10:33:13.968921 139647375513408 <ipython-input-43-1a70036edb06>:35]   Num examples = 6391
I1202 10:33:13.969562 139647375513408 <ipython-input-43-1a70036edb06>:36]   Batch size = 32


...

I1202 11:48:35.542763 139647375513408 modeling_utils.py:205] Model weights saved in outputs/transformers/binary-bce/checkpoint-27000/pytorch_model.bin
I1202 11:48:35.543758 139647375513408 <ipython-input-45-08f342e6a116>:169] Saving model checkpoint to outputs/transformers/binary-bce/checkpoint-27000


Epoch: 100%|██████████| 3/3 [1:41:21<00:00, 2028.21s/it]
I1202 11:54:12.752272 139647375513408 <ipython-input-40-fa36fc6f9a16>:14] Loading features from cached file cache/transformers/cached_dev_distilbert-base-uncased_512_binary-bce


I1202 11:54:13.070953 139647375513408 <ipython-input-43-1a70036edb06>:34] ***** Running evaluation  *****
I1202 11:54:13.071983 139647375513408 <ipython-input-43-1a70036edb06>:35]   Num examples = 6391
I1202 11:54:13.072525 139647375513408 <ipython-input-43-1a70036edb06>:36]   Batch size = 32


I1202 11:55:13.198882 139647375513408 <ipython-input-43-1a70036edb06>:105] ***** Eval results  *****
I1202 11:55:13.199559 139647375513408 <ipython-input-43-1a70036edb06>:108]   acc = 0.8166171178219371
I1202 11:55:13.200152 139647375513408 <ipython-input-43-1a70036edb06>:108]   f1 = 0.8080576482148705
I1202 11:55:13.200648 139647375513408 <ipython-input-43-1a70036edb06>:108]   fn = 965
I1202 11:55:13.201112 139647375513408 <ipython-input-43-1a70036edb06>:108]   fp = 207
I1202 11:55:13.201574 139647375513408 <ipython-input-43-1a70036edb06>:108]   mcc = 0.6558804390511694
I1202 11:55:13.202028 139647375513408 <ipython-input-43-1a70036edb06>:108]   tn = 2752
I1202 11:55:13.202485 139647375513408 <ipython-input-43-1a70036edb06>:108]   tp = 2467
I1202 11:55:13.203050 139647375513408 <ipython-input-46-e48efd1a75ca>:16]  global_step = 28758, average loss = 0.37328667078631256



Time for [3 - train (fine-tune) model]: 1:42:26.806061


In [47]:
# saving
if args['do_train']:
    if not os.path.exists(args['output_dir']):
        os.makedirs(args['output_dir'])
    
    logger.info("Saving model checkpoint to %s", args['output_dir'])

    model_to_save = model.module if hasattr(
        model,
        'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(args['output_dir'])
    tokenizer.save_pretrained(args['output_dir'])
    torch.save(args, os.path.join(args['output_dir'], 'training_args.bin'))

I1202 12:27:10.138892 139647375513408 <ipython-input-47-94c486dab606>:6] Saving model checkpoint to outputs/transformers/binary-bce
I1202 12:27:10.141351 139647375513408 configuration_utils.py:71] Configuration saved in outputs/transformers/binary-bce/config.json
I1202 12:27:10.308016 139647375513408 modeling_utils.py:205] Model weights saved in outputs/transformers/binary-bce/pytorch_model.bin


In [48]:
with open(os.path.join(args['output_dir'], 'done.flag'), "w") as fp:
    fp.write("Done.")

In [52]:
# loading
model_dir = args['output_dir']  # this should probably be somehow set to load the args/config
args = torch.load(os.path.join(model_dir, 'training_args.bin'))

config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]

# is not saved?
config = config_class.from_pretrained(args['model_name'],
                                      num_labels=args['num_labels'],
                                      finetuning_task=args['task_name'])

# tokenizer.save_pretrained(args['output_dir'])
tokenizer = tokenizer_class.from_pretrained(args['output_dir'])

# model_to_load = model.module if hasattr(
#     model,
#     'module') else model  # Take care of distributed/parallel training
# model_to_save.save_pretrained(args['output_dir'])
model = model_class.from_pretrained(args['output_dir'], num_labels=args['num_labels'])

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

I1202 12:30:09.601337 139647375513408 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at /home/ekoerner/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I1202 12:30:09.603906 139647375513408 configuration_utils.py:168] Model config {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "binary-bce",
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "n_heads": 12,
  "n_layers": 6,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1202 

DistilBertForSameSideBCEClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (dropout): Dropout(p=0.1)
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropou

In [53]:
_ = evaluate(model, tokenizer, args, ds_train, ds_dev)

I1202 12:30:32.093506 139647375513408 <ipython-input-40-fa36fc6f9a16>:14] Loading features from cached file cache/transformers/cached_dev_distilbert-base-uncased_512_binary-bce
I1202 12:30:32.740238 139647375513408 <ipython-input-43-1a70036edb06>:34] ***** Running evaluation  *****
I1202 12:30:32.741215 139647375513408 <ipython-input-43-1a70036edb06>:35]   Num examples = 6391
I1202 12:30:32.741795 139647375513408 <ipython-input-43-1a70036edb06>:36]   Batch size = 32


I1202 12:31:31.700134 139647375513408 <ipython-input-43-1a70036edb06>:105] ***** Eval results  *****
I1202 12:31:31.700829 139647375513408 <ipython-input-43-1a70036edb06>:108]   acc = 0.8166171178219371
I1202 12:31:31.701335 139647375513408 <ipython-input-43-1a70036edb06>:108]   f1 = 0.8080576482148705
I1202 12:31:31.701820 139647375513408 <ipython-input-43-1a70036edb06>:108]   fn = 965
I1202 12:31:31.702291 139647375513408 <ipython-input-43-1a70036edb06>:108]   fp = 207
I1202 12:31:31.702757 139647375513408 <ipython-input-43-1a70036edb06>:108]   mcc = 0.6558804390511694
I1202 12:31:31.703285 139647375513408 <ipython-input-43-1a70036edb06>:108]   tn = 2752
I1202 12:31:31.703779 139647375513408 <ipython-input-43-1a70036edb06>:108]   tp = 2467


In [49]:
# reverse argument order
# --> results almost seems the same!

ds_dev = df2ds_rev(X_dev, y_dev)
# remove cached dev file
_ = evaluate(model, tokenizer, args, ds_train, ds_dev)

I1202 12:27:28.669451 139647375513408 <ipython-input-40-fa36fc6f9a16>:14] Loading features from cached file cache/transformers/cached_dev_distilbert-base-uncased_512_binary-bce
I1202 12:27:29.013592 139647375513408 <ipython-input-43-1a70036edb06>:34] ***** Running evaluation  *****
I1202 12:27:29.014484 139647375513408 <ipython-input-43-1a70036edb06>:35]   Num examples = 6391
I1202 12:27:29.015063 139647375513408 <ipython-input-43-1a70036edb06>:36]   Batch size = 32


I1202 12:28:27.800076 139647375513408 <ipython-input-43-1a70036edb06>:105] ***** Eval results  *****
I1202 12:28:27.800774 139647375513408 <ipython-input-43-1a70036edb06>:108]   acc = 0.8166171178219371
I1202 12:28:27.801353 139647375513408 <ipython-input-43-1a70036edb06>:108]   f1 = 0.8080576482148705
I1202 12:28:27.801861 139647375513408 <ipython-input-43-1a70036edb06>:108]   fn = 965
I1202 12:28:27.802338 139647375513408 <ipython-input-43-1a70036edb06>:108]   fp = 207
I1202 12:28:27.802808 139647375513408 <ipython-input-43-1a70036edb06>:108]   mcc = 0.6558804390511694
I1202 12:28:27.803338 139647375513408 <ipython-input-43-1a70036edb06>:108]   tn = 2752
I1202 12:28:27.803905 139647375513408 <ipython-input-43-1a70036edb06>:108]   tp = 2467


In [ ]:
results, wrong = evaluate(model, tokenizer, args, ds_train, ds_dev)

In [ ]:
preds, out_label_ids = get_train_output(model, tokenizer, args, ds_train, ds_dev, prefix="", evaluate=True)

In [ ]:
np.unique(preds, return_counts=True)

In [ ]:
labels = out_label_ids
confusion_matrix(labels, preds).ravel()

In [ ]:
vals = np.array([-1.0, -0.1, 0.1, 0.4, 0.5, 0.6, 1.0, 2.0])
vals_s = sigmoid(vals)
vals_s_r = vals_s.round()

vals, vals_s, vals_s_r

In [ ]:
jfkdöljfkdlasöjfkld
# abort here

---

In [ ]:
if False:
    # BERT MXNet setup
    def setup_bert():
        # change `ctx` to `mx.cpu()` if no GPU is available.
        ctx = mx.gpu(0)
        # ctx = [mx.gpu(i) for i in range(2)]
        # ctx =  mx.gpu() if mx.context.num_gpus() else mx.cpu()
        # ctx = mx.cpu()

        bert_base, vocabulary = nlp.model.get_model(
            'bert_12_768_12',
            dataset_name='book_corpus_wiki_en_uncased',
            pretrained=True,
            ctx=ctx,
            use_pooler=True,
            use_decoder=False,
            use_classifier=False)
        print(bert_base)

        #model = BERTProEpiClassifier(bert_base, num_classes=2, dropout=0.1)
        model = BERTProEpiClassifier(bert_base, num_classes=1, dropout=0.1)
        # only need to initialize the classifier layer.
        model.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
        model.hybridize(static_alloc=True)

        # softmax cross entropy loss for classification
        #loss_function = gluon.loss.SoftmaxCELoss()
        loss_function = gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=False)
        loss_function.hybridize(static_alloc=True)

        metric = mx.metric.Accuracy()

        # use the vocabulary from pre-trained model for tokenization
        bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)
        # maximum sequence length
        # max_len = 128  # + batch_size: 32
        # 384 - 12
        max_len = 512  # + batch_size: 6 ?
        # the labels for the two classes
        #all_labels = ["0", "1"]
        all_labels = [0, 1]
        # whether to transform the data as sentence pairs.
        # for single sentence classification, set pair=False
        transform = FirstAndLastPartBERTDatasetTransform(bert_tokenizer,
                                                         max_len,
                                                         labels=all_labels,
                                                         label_dtype='int32',
                                                         pad=True,
                                                         pair=True)

        return model, vocabulary, ctx, bert_tokenizer, transform, loss_function, metric, all_labels

In [ ]:
if False:
    # BERT MXNet train
    def train(model,
              data_train,
              ctx,
              metric,
              loss_function,
              batch_size=32,
              lr=5e-6,
              num_epochs=3,
              sw=None,
              checkpoint_dir="data",
              use_checkpoints=True):
        with Timer("setup training"):
            train_sampler = nlp.data.FixedBucketSampler(
                lengths=[int(item[1]) for item in tqdm(data_train)],
                batch_size=batch_size,
                shuffle=True)
            bert_dataloader = mx.gluon.data.DataLoader(data_train,
                                                       batch_sampler=train_sampler)

            trainer = gluon.Trainer(model.collect_params(), 'adam', {
                'learning_rate': lr,
                'epsilon': 1e-9
            })

            # collect all differentiable parameters
            # grad_req == 'null' indicates no gradients are calculated (e.g. constant parameters)
            # the gradients for these params are clipped later
            params = [
                p for p in model.collect_params().values() if p.grad_req != 'null'
            ]

        log_interval = 500
        global_step = 0
        with Timer("training"):
            stats = list()
            for epoch_id in range(num_epochs):
                if use_checkpoints:
                    epoch_checkpoint_savefile = "bert.model.checkpoint{}.params".format(
                        epoch_id)
                    if checkpoint_dir is not None:
                        epoch_checkpoint_savefile = os.path.join(
                            checkpoint_dir, epoch_checkpoint_savefile)
                    if os.path.exists(epoch_checkpoint_savefile):
                        model.load_parameters(epoch_checkpoint_savefile, ctx=ctx)
                        print("loaded checkpoint for epoch {}".format(epoch_id))
                        continue

                with Timer("epoch {}".format(epoch_id)):
                    metric.reset()
                    step_loss = 0
                    global_step = epoch_id * len(bert_dataloader)
                    t_p = time.time()  # time keeping
                    for batch_id, (token_ids, valid_length, segment_ids,
                                   token_ids_epi, valid_length_epi,
                                   segment_ids_epi,
                                   label) in enumerate(tqdm(bert_dataloader)):
                        global_step += 1
                        with mx.autograd.record():
                            # load data to GPU
                            token_ids = token_ids.as_in_context(ctx)
                            valid_length = valid_length.as_in_context(ctx)
                            segment_ids = segment_ids.as_in_context(ctx)
                            token_ids_epi = token_ids_epi.as_in_context(ctx)
                            valid_length_epi = valid_length_epi.as_in_context(ctx)
                            segment_ids_epi = segment_ids_epi.as_in_context(ctx)
                            label = label.as_in_context(ctx)

                            # forward computation
                            out = model(token_ids, segment_ids,
                                        valid_length.astype('float32'),
                                        token_ids_epi, segment_ids_epi,
                                        valid_length_epi.astype('float32'))
                            label = label.astype('float32')
                            ls = loss_function(out, label).mean()

                        # backward computation
                        ls.backward()

                        # gradient clipping
                        trainer.allreduce_grads()
                        nlp.utils.clip_grad_global_norm(params, 1)
                        trainer.update(1)

                        step_loss += ls.asscalar()
                        out = out.sigmoid().round().astype('int32')
                        label = label.astype('int32')
                        metric.update([label], [out])
                        stats.append((metric.get()[1], ls.asscalar()))

                        if sw:
                            sw.add_scalar(tag='T-ls', value=ls.asscalar(), global_step=global_step)
                            sw.add_scalar(tag='T-acc', value=metric.get()[1], global_step=global_step)

                        if (batch_id + 1) % (log_interval) == 0:
                            print(
                                '[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f} - time {}'
                                .format(
                                    epoch_id, batch_id + 1, len(bert_dataloader),
                                    step_loss / log_interval,
                                    trainer.learning_rate,
                                    metric.get()[1],
                                    datetime.timedelta(seconds=(time.time() -
                                                                t_p))))
                            t_p = time.time()
                            step_loss = 0

                if use_checkpoints:
                    model.save_parameters(epoch_checkpoint_savefile)

        return stats

In [ ]:
if False:
    # BERT MXNet predict
    def predict_unknown(model, data_predict, ctx, label_map=None, batch_size=32):
        bert_dataloader = mx.gluon.data.DataLoader(data_predict,
                                                   batch_size=batch_size)

        predictions = list()

        with Timer("prediction"):
            for batch_id, (token_ids, valid_length, segment_ids, token_ids_epi,
                           valid_length_epi,
                           segment_ids_epi) in enumerate(tqdm(bert_dataloader)):
                global_step = batch_id
                # load data to GPU
                token_ids = token_ids.as_in_context(ctx)
                valid_length = valid_length.as_in_context(ctx)
                segment_ids = segment_ids.as_in_context(ctx)
                token_ids_epi = token_ids_epi.as_in_context(ctx)
                valid_length_epi = valid_length_epi.as_in_context(ctx)
                segment_ids_epi = segment_ids_epi.as_in_context(ctx)

                # forward computation
                out = model(token_ids, segment_ids, valid_length.astype('float32'),
                            token_ids_epi, segment_ids_epi,
                            valid_length_epi.astype('float32'))

                # to binary: 0/1
                out = out.sigmoid().round().astype('int32')
                # to numpy (not mxnet)
                out = out.asnumpy()
                # get mapping type
                if label_map:
                    out = [label_map[c] for c in list(out)]

                predictions.extend(out)

        # list to numpy array
        predictions = np.array(predictions)

        return predictions

---